# Setting Jubilee Tool Parking Positions
This notebook scaffolds the steps outlined [here](https://www.jubilee3d.com/index.php?title=Setting_Tool_Parking_Positions) to set tool parking positions from the Jubilee wiki. Some of these steps are manual, and others require sending commands. This notebook will walk through all necessary steps!

### 0. Notebook Setup
We'll need to send commands to the machine-- the following cells set up this communication. Be sure to have a usb cable connecting your duet board to your machine.

In [2]:
# import required modules
from science_jubilee.Machine import Machine
import os
import sys
from jinja2 import Environment, FileSystemLoader, select_autoescape

In [3]:
# Establish connection with machine
m = Machine(address="jubilee.local") # edit the address as necessary

In [ ]:
tool_number = 2           # What is the number of the tool you are setting up? 
tool_name = "Side Camera" # Set a human-readable name for this tool

# Pickup and park moves should be 'manhattan' style to avoid collisions
# That is, we should move in a square block (first X then Y) rather than along the shortest path
# A default value of 60 usually works well here!
manhattan_offset = 60 

### 1. Tool Definition
We need to tell the Duet some information about the tool we are going to set up. Run the following cell and copy the output into the end of your config.g file (System-->config.g in DuetWebControl). Note that additional edits to config.g might be necessary for your specific tool; see the documentation page for your tool for more information.

In [ ]:
print(f'M563 P{tool_number} S"{tool_name}"')

### 2. Coursely place your parking post.
Take your parking post assembly and loosely fasten it to the rail approximately where you want it; you should still be able to move the parking post around to adjust it for X-axis position and parking post height.

## 3. Move the build plate
Move your build plate down in the Z-axis (positive Z values) such that any tools you’re about to mount will fully clear the bed. Usually a value of z=50 or more is enough for this-- you can edit the value in the cell below to accommodate longer tools.

In [ ]:
m.home_all() # home if needed; make sure your build plate is clear!
m.move_to(z=150)

### 4. Jog the carriage to somewhere close to the center of the printer bed.

In [ ]:
m.move_to(x=150, y=150)

### 5. With no active tool, manually place your tool against the lock carriage, and lock the tool onto the carriage using the tool lock macro (Do not select a tool using the Tx commands - this is critical)
Run the following tool lock cell once you have placed the tool against the lock carriage.

In [ ]:
m.tool_lock()

### 6. Jog the carriage to roughly where you want to end up across the X-axis. Eyeball it, or use a measuring tape/ruler if you want to.
You can send coordinates below, otherwise the DuetWebControl dashboard is useful for this step.

In [ ]:
# m.move_to(x=283.3)

### 6. Jog the carriage forward in the Y-axis until its just in front of the parking post assembly, making sure you don’t engage the parking dowel pins just yet.
Again, controls from DuetWebControl are useful here

In [ ]:
# m.move_to(y=<approx-position>)

### 8. Line up the parking post on the rail to the tool across the X-axis.
You can do this by physically moving the parking post along the rail

### 9. Slowly jog the carriage forward in the Y-axis until the tool starts engaging the parking dowel pins.
You can do this by incrementing by y=+1 from DuetWebControl.

### 10. Take a look to make sure everything is lined up: dowel pins, parking wings, etc.

### 11. Jog the carriage further in the Y-axis in small (1mm) increments until you’ve achieved a full dock. Depending on your tool mount, you may or may not have the tool actually touching the front of the extrusion that holds the parking posts.
You can do this by incrementing by y=+1 from the DuetWebControl

### 12. Adjust the position in the X-axis to your liking (not by hand, we don’t want to lose position).
You can do this by incrementing by x=+/-0.1 from DuetWebControl; right-click the relevant button if you'd like to add a smaller step value.

### 13. Once your tool and parking post are fully engaged and in the final position you want them to be, take a note of the X and Y values reported for the printer’s current position in the web interface. This will be needed for the tpost and tfree macros for this specific tool and parking position.

In [ ]:
pos = m.get_position()
x_park = float(pos['X'])
y_park = float(pos['Y'])
x_park, y_park

### 14. Tighten down the tee-nuts to lock in your X-axis position for this tool.

### 15. Adjust your parking post height adjustment until you’re satisfied that the tool is properly supported and parallel to the Y-axis rails of your printer. Tighten down the screws for the height adjustment to lock it in.

### 16. Unlock the tool from the carriage using the tool unlock macro (and not using a T-1 or Tx command).

In [ ]:
m.tool_unlock()

### 17. Jog the now-decoupled carriage backwards in the Y-axis direction until it fully clears your tool(s) and parking post(s). Take a note of this value of the Y-axis position from the web interface, as this will be close to your final choice of “Tool Clearance Limit” for all of your tfree files.
Duet3 console controls can be used here; you can run the following cell once the carriage is clear

In [ ]:
y_clear = float(m.getPosition()['Y'])
y_clear

### 18. Update the relevant tpost and tfree files for the tool you have just set up. 
There are 3 files which need to be updated with the values you've found. We'll use template files to make this quicker and more reliable-- run the following cell to set up the templating engine. 

In [ ]:
env = Environment(loader= FileSystemLoader("templates"))

The "Txpost.g" file is called for tool number 'x' after the firmware thinks Toolx is selected. From the duet console, navigate to system-->Txpost.g, where x is the number of the tool you are currently calibrating-- create the file if it doesn't yet exist. Run the following cell and replace the contents (i.e. copy-paste) with the output produced:

In [ ]:
template = env.get_template("tpost.g")
tpost_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpost_content)

Similarly, tfreex.g uns at the start of a toolchange if the current tool is tool-x. Again, copy-paste the following output into tfreex.g:

In [ ]:
template = env.get_template("tfree.g")
tfree_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tfree_content)

Finally, the tprex.g file is where the carriage will move to immediately before & after picking up a tool. It is recommended that all pick-up/drop-off moves are 'Manhattan' style (a big negative move in the Y-axis until you’re completely clear of any tools and parking posts, then a move in the X-axis) to avoid collisions with other tools or parking posts). We use a default value of 60mm. Replace tpre.x with the following contents:

In [ ]:
template = env.get_template("tpre.g")
tpre_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpre_content)

### 19. Try out a tool change to the new parking post using the Tx command!
Test both selecting a tool and parking it. While testing, check to see if your setup needs a few tweaks to make sure the lock pin fully engages the ramp (change the Y-axis value used in tpost - higher Y values move the locking carriage closer to the tool dock) upon locking. Check to see if the tool has been docked correctly after unlocking (change the Y-axis values in tfree to adjust this to your needs).

In [ ]:
m.toolChange(tool_number)

In [ ]:
m.toolChange(-1)